In [1]:
import pickle
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#importación dataset
df22=pd.read_csv('../data/players_22.csv')


C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Limpieza datos

In [3]:
 df22['año_version']=2022

In [4]:
df22['años_de_contrato']=np.where((df22['club_contract_valid_until']-df22['año_version'])<0,0,(df22['club_contract_valid_until']-df22['año_version']))
df22.groupby('años_de_contrato')['años_de_contrato'].count()

años_de_contrato
0.0    9660
1.0    4545
2.0    3092
3.0    1429
4.0     435
5.0      15
6.0       1
9.0       1
Name: años_de_contrato, dtype: int64

In [5]:
#eliminamos columnas de url que no aportan información
df22 = df22[df22.columns.drop(df22.filter(regex='url').columns)]

In [6]:
#funcion transformación en lista
def lista_posicion(x):
    return (x.replace(' ','').split(','))

In [7]:
#transforma valores de columna player_positions en una lista
df22['player_positions']=df22['player_positions'].apply(lambda x: lista_posicion(x))

In [8]:
#defino listas para agrupar posiciones por 4 categorias
pos_defensas=['LCB','RCB','CB','RB','LB','RWB','LWB']
pos_medios=['LW','LCM','RCM','CAM','RW','LDM','CDM','CM','RDM','RM','RAM','LAM','LM']
pos_delanteros=['CF','ST','RS','LS','RF']

In [9]:
#agrupo posiciones en posiciones genéricas    
for i in df22.index:
            if len(df22.at[i,'player_positions'])==1:
                if df22.at[i,'player_positions'][0]=='GK':
                    df22.at[i,'player_positions']='portero'
                if df22.at[i,'player_positions'][0] in pos_defensas:
                    df22.at[i,'player_positions']='defensa'
                elif df22.at[i,'player_positions'][0] in pos_medios:
                    df22.at[i,'player_positions']='centrocampista'
                elif df22.at[i,'player_positions'][0] in pos_delanteros:
                    df22.at[i,'player_positions']='delantero'
            else:
                if df22.at[i,'player_positions'][0] in pos_defensas:
                    df22.at[i,'player_positions']='defensa'
                elif df22.at[i,'player_positions'][0] in pos_medios:
                    df22.at[i,'player_positions']='centrocampista'
                elif df22.at[i,'player_positions'][0] in pos_delanteros:
                    df22.at[i,'player_positions']='delantero'
                elif df22.at[i,'player_positions'][0]=='GK':
                    df22.at[i,'player_positions']='portero'

In [10]:
#filtramos dataset 
df22=df22[df22['player_positions']=='portero']

In [11]:
#Agrupa por Equipo y año versión(temporada) y añade el total en cada jugador
df22['overall_team/year'] = df22.groupby('club_team_id')['overall'].transform('sum')

In [12]:
#Crea columna nº de equipos por liga/año
df22['teams_in_league/year'] = df22.groupby('league_name')['club_team_id'].transform('nunique')

In [13]:
#Crea columna media liga/año
df22['mean_league/year'] = df22.groupby('league_name')['overall'].transform('mean')

In [14]:
#Crea columna media equipo/año
df22['mean_team/year'] = df22.groupby('club_team_id')['overall'].transform('mean')

In [15]:
#df solo con las columnas a transformar
posiciones=df22[["ls","st","rs","lw","lf","cf","rf","rw", 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm',
     'lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb',"gk"]]

In [16]:
#funcion para limpiar valores
def limp_posiciones(x):
    x.replace("+"," ")
    return x.split(" ")[0]

In [17]:
#limpiar en df
for i in posiciones.columns:
    df22[i]=df22[i].astype(str)
    df22[i]=df22[i].apply(limp_posiciones)

In [18]:
df22['lf'] = df22['lf'].str.replace(r'\+', ' ', regex=True)
df22['cf'] = df22['cf'].str.replace(r'\+', ' ', regex=True)
df22['rf'] = df22['rf'].str.replace(r'\+', ' ', regex=True)
df22['lam'] = df22['lam'].str.replace(r'\+', ' ', regex=True)
df22['ram'] = df22['ram'].str.replace(r'\+', ' ', regex=True)
df22['lm'] = df22['lm'].str.replace(r'\+', ' ', regex=True)
df22['rm'] = df22['rm'].str.replace(r'\+', ' ', regex=True)
df22['cam'] = df22['cam'].str.replace(r'\+', ' ', regex=True)
df22['rs'] = df22['rs'].str.replace(r'\+', ' ', regex=True)
df22['gk'] = df22['gk'].str.replace(r'\+', ' ', regex=True)

df22['lf'] = df22['lf'].str.replace(r'\-', ' ', regex=True)
df22['cf'] = df22['cf'].str.replace(r'\-', ' ', regex=True)
df22['rf'] = df22['rf'].str.replace(r'\-', ' ', regex=True)
df22['lam'] = df22['lam'].str.replace(r'\-', ' ', regex=True)
df22['ram'] = df22['ram'].str.replace(r'\-', ' ', regex=True)
df22['lm'] = df22['lm'].str.replace(r'\-', ' ', regex=True)
df22['rm'] = df22['rm'].str.replace(r'\-', ' ', regex=True)
df22['cam'] = df22['cam'].str.replace(r'\-', ' ', regex=True)
df22['gk'] = df22['gk'].str.replace(r'\-', ' ', regex=True)

In [19]:
#funcion para limpiar valores
def limp_posiciones(x):
    return x.split(" ")[0]

In [20]:
for i in ['lf','cf','rf','rs','lam','ram','cam','lm','rm','gk']:
    df22[i]=df22[i].astype(str)
    df22[i]=df22[i].apply(limp_posiciones)

In [21]:
df22[['lf','cf','rf','rs','lam','ram','cam','lm','rm','gk']]

,lf,cf,rf,rs,lam,ram,cam,lm,rm,gk
5,35,35,35,33,38,38,38,35,35,89
7,43,43,43,40,47,47,47,44,44,88
8,38,38,38,35,42,42,42,39,39,88
12,31,31,31,31,32,32,32,31,31,86
18,43,43,43,40,47,47,47,44,44,87
...,...,...,...,...,...,...,...,...,...,...
19210,17,17,17,19,17,17,17,16,16,47
19216,18,18,18,18,19,19,19,19,19,47
19225,19,19,19,19,20,20,20,19,19,46
19227,17,17,17,17,18,18,18,18,18,46


## Definir X

In [22]:
#Variables a usar (según notebook de modelo):
variables=['overall',
 'value_eur',
 'wage_eur',
 'age',
 'height_cm',
 'weight_kg',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_reactions',
 'movement_balance',
 'power_shot_power',
 'power_jumping',
 'power_stamina',
 'power_strength',
 'power_long_shots',
 'mentality_aggression',
 'mentality_interceptions',
 'mentality_positioning',
 'mentality_vision',
 'mentality_penalties',
 'mentality_composure',
 'defending_marking_awareness',
 'defending_standing_tackle',
 'goalkeeping_kicking',
 'gk',
 'overall_team/year',
 'teams_in_league/year',
 'mean_league/year',
 'años_de_contrato']

In [23]:
X22=df22[variables]

In [24]:
#llenar valor nulos de forma preventiva
X22.fillna(0,inplace=True)

C:\Users\carlo\anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [25]:
#transformar datos a numérico
X22['gk']=X22.gk.astype(int)

<ipython-input-25-01dbb10a3d63>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X22['gk']=X22.gk.astype(int)


## aplicar modelo

In [26]:
with open('../models/model_GK.pkl','rb') as f:
    super_modelo=pickle.load(f)

In [27]:
y_pred=super_modelo.predict(X22)

In [28]:
ind_list=X22[y_pred==1].index.to_list()

In [29]:
#top 5 porteros con mayor proyección
df22[df22.index.isin(ind_list)].sort_values(by='overall', ascending=False).head()

,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,...,cb,rcb,rb,gk,año_version,años_de_contrato,overall_team/year,teams_in_league/year,mean_league/year,mean_team/year
15970,207589,M. Säfqvist,Malkolm Johan Henrik Nilsson Säfqvist,portero,59,62,220000.0,650.0,27,1993-08-03,...,26+2,26+2,23+2,58,2022,0.0,115.0,16.0,61.093023,57.500000
16041,236794,J. Dakir,Jonas Dakir,portero,59,63,250000.0,1000.0,24,1997-04-18,...,23+2,23+2,22+2,58,2022,0.0,188.0,12.0,62.593750,62.666667
15981,215491,K. Rosa,Kacper Rosa,portero,59,63,240000.0,1000.0,26,1994-08-13,...,24+2,24+2,24+2,58,2022,0.0,252.0,18.0,61.333333,63.000000
16448,263441,L. Bonelli,Lucas Bonelli,portero,59,74,450000.0,500.0,18,2003-02-14,...,23+2,23+2,22+2,58,2022,1.0,274.0,20.0,69.267606,68.500000
16396,262157,M. Mignon,Maxime Mignon,portero,59,59,150000.0,2000.0,30,1990-08-17,...,20+2,20+2,17+2,58,2022,0.0,178.0,18.0,64.732143,59.333333
